<a href="https://colab.research.google.com/github/cheaqingqing/ADSP/blob/main/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# Step 1: Install required library
!pip install openpyxl

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/ADSP Final Project/student_data.xlsx'

# Step 5: Try to load existing Excel file, else create new DataFrame
try:
    students_data = pd.read_excel(file_path)
except FileNotFoundError:
    # Ensure consistent column names if a new DataFrame is created
    students_data = pd.DataFrame(columns=["ID", "First Name", "Last Name", "Total Score"])

# Step 6: Get user input
student_id = input("Enter Student ID: ")
first_name = input("Enter First Name: ")
last_name = input("Enter Last Name: ")
total_score = input("Enter Total Score: ")

# Step 7: Append new student record
# Adjust new_record keys to match existing DataFrame columns
new_record = {
    "ID": student_id,
    "First Name": first_name,
    "Last Name": last_name,
    "Total Score": total_score
}

# Convert new_record to a DataFrame and use pd.concat
students_data = pd.concat([students_data, pd.DataFrame([new_record])], ignore_index=True)

# Step 8: Save back to Excel
students_data.to_excel(file_path, index=False)

print("✅ Student record saved successfully to Excel in Google Drive!")

Enter Student ID: 1
Enter First Name: Chea
Enter Last Name: Chingching
Enter Total Score: 12
✅ Student record saved successfully to Excel in Google Drive!


In [ ]:
class Node:
    def __init__(self, student_id, first_name, last_name, total_score):
        self.student_id = student_id
        self.first_name = first_name
        self.last_name = last_name
        self.total_score = total_score
        self.left = None
        self.right = None

In [ ]:
# =========================
# BST Class
# =========================
class BST:
    def __init__(self):
        self.root = None

    def insert(self, root, student_id, first_name, last_name, total_score):
        if root is None:
            return Node(student_id, first_name, last_name, total_score)
        if student_id < root.student_id:
            root.left = self.insert(root.left, student_id, first_name, last_name, total_score)
        elif student_id > root.student_id:
            root.right = self.insert(root.right, student_id, first_name, last_name, total_score)
        return root

    def inorder(self, root):
        if root:
            self.inorder(root.left)
            print(f"ID:{root.student_id}, Name:{root.first_name} {root.last_name}, Score:{root.total_score}")
            self.inorder(root.right)

    def search(self, root, student_id):
        if root is None or root.student_id == student_id:
            return root
        if student_id < root.student_id:
            return self.search(root.left, student_id)
        return self.search(root.right, student_id)

    def delete(self, root, student_id):
        if root is None:
            return root
        if student_id < root.student_id:
            root.left = self.delete(root.left, student_id)
        elif student_id > root.student_id:
            root.right = self.delete(root.right, student_id)
        else:
            # Node found
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            # Find inorder successor
            temp = self.minValueNode(root.right)
            root.student_id, root.first_name, root.last_name, root.total_score = (
                temp.student_id, temp.first_name, temp.last_name, temp.total_score
            )
            root.right = self.delete(root.right, temp.student_id)
        return root

    def minValueNode(self, node):
        current = node
        while current.left is not None:
            current = current.left
        return current

In [ ]:
try:
    students_data = pd.read_excel(file_path)
except FileNotFoundError:
    students_data = pd.DataFrame(columns=["Student_ID", "First_Name", "Last_Name", "Total_Score"])

In [ ]:

bst = BST()
root = None

# Insert existing records into BST
for _, row in students_data.iterrows():
    root = bst.insert(root, int(row["ID"]), row["First Name"], row["Last Name"], int(row["Total Score"]))

In [ ]:
while True:
    print("\n===== Student BST Menu =====")
    print("1. Display Info")
    print("2. Add Info")
    print("3. Remove Info")
    print("4. Update Info")
    print("5. Exit")

    choice = input("Enter choice: ")

    if choice == "1":
        print("\n--- Student Records ---")
        bst.inorder(root)

    elif choice == "2":
        sid = int(input("Enter Student ID: "))
        fname = input("Enter First Name: ")
        lname = input("Enter Last Name: ")
        score = int(input("Enter Total Score: "))
        root = bst.insert(root, sid, fname, lname, score)
        students_data = pd.concat([students_data, pd.DataFrame([{
            "Student_ID": sid, "First_Name": fname, "Last_Name": lname, "Total_Score": score
        }])], ignore_index=True)
        students_data.to_excel(file_path, index=False)
        print("✅ Student added successfully!")

    elif choice == "3":
        sid = int(input("Enter Student ID to remove: "))
        root = bst.delete(root, sid)
        students_data = students_data[students_data["Student_ID"] != sid]
        students_data.to_excel(file_path, index=False)
        print("✅ Student removed successfully!")

    elif choice == "4":
        sid = int(input("Enter Student ID to update: "))
        node = bst.search(root, sid)
        if node:
            fname = input(f"Enter new First Name (current: {node.first_name}): ")
            lname = input(f"Enter new Last Name (current: {node.last_name}): ")
            score = int(input(f"Enter new Total Score (current: {node.total_score}): "))
            node.first_name, node.last_name, node.total_score = fname, lname, score
            students_data.loc[students_data["Student_ID"] == sid, ["First_Name", "Last_Name", "Total_Score"]] = [fname, lname, score]
            students_data.to_excel(file_path, index=False)
            print("✅ Student updated successfully!")
        else:
            print("❌ Student not found!")

    elif choice == "5":
        print("Exiting program...")
        break

    else:
        print("Invalid choice, try again.")


===== Student BST Menu =====
1. Display Info
2. Add Info
3. Remove Info
4. Update Info
5. Exit
Enter choice: 5
Exiting program...
